In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.4/402.4 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.7 MB/s eta 0:00:0000:0100:01m
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 9.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 7.6 MB/s eta 0:00:0000:0100:

In [5]:
base_dir = '/Users/mehrad/research/self/codes/data'

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, StorageContext, SimpleKeywordTableIndex
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KeywordTableSimpleRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.schema import NodeWithScore
from llama_index.core.postprocessor import SimilarityPostprocessor
import torch
import json

import pickle

In [4]:
with open("/Users/mehrad/research/self/codes/data/battles_text_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [6]:
for battle in data:
  with open(f"{base_dir}/docs/{battle['title']}.json", "w") as f:
    json.dump(battle, f)

In [7]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 192
Settings.chunk_overlap = 24

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [8]:
reader = SimpleDirectoryReader(f"{base_dir}/docs/")
documents = reader.load_data()

In [9]:
print(document)
len(documents)

456

In [11]:
keyword_index = SimpleKeywordTableIndex.from_documents(documents)

In [44]:
#query = "who won the siege of osaka?"
query = input()

In [45]:
keyword_retriever = KeywordTableSimpleRetriever(keyword_index, similarity_top_k=15)

In [48]:
tmp = keyword_retriever.retrieve(query)
tops = []
for x in tmp:
  tops.append({query:x.text})
print(tops)

In [50]:
for i,x in enumerate(tops):
  with open(f"/Users/mehrad/research/self/codes/data/topdocs/{i}.json", "w") as f:
      json.dump(x, f)

In [51]:
vsi = SimpleDirectoryReader("/Users/mehrad/research/self/codes/data//topdocs")
top_docs = vsi.load_data()
vsi_index = VectorStoreIndex.from_documents(top_docs)

In [53]:
retriever = VectorIndexRetriever(
    index=vsi_index,
    similarity_top_k=5,
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
)

In [55]:
response = query_engine.query(query)

In [60]:
context_texts = [x.text for x in response.source_nodes]
context = "Context:\n"
for i in range(min(5, len(response.source_nodes))):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
{"who won the siege of osaka?": "{\"title\": \"Siege of Akasaka\", \"paragraphs\": \"The  siege of Akasaka  was one of the earlier battles of the  Genk\\u014d War  between the figurehead  Emperor   Godaigo  and the largely  H\\u014dj\\u014d -controlled  Kamakura shogunate  during the final years of the  Kamakura period  in  Japan .

{"who won the siege of osaka?": "Hideyoshi's grave, along with Kyoto's  Toyokuni Shrine , were destroyed subsequently during the Tokugawa shogunate. Ch\\u014dsokabe Morichika was beheaded on May 11. There are also records of pillaging and mass rapes by Tokugawa forces at the closing of the siege. [ citation needed ] The  bakufu  obtained 650,000  koku  at  Osaka  and started rebuilding Osaka Castle. Osaka was then made a  han  (feudal domain), and given to  Matsudaira Tadayoshi ."}

{"who won the siege of osaka?": "ISBN \\u00a0 462307689X . Sankyoden Editorial Office; Watanabe Seisuke (1982).  \\u6bdb\\u5229\\u8f1d\\u5143\\u537f\\u4f1d . \\u30de\\u

In [233]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/xLAM-1b-fc-r")
model = AutoModelForCausalLM.from_pretrained("Salesforce/xLAM-1b-fc-r")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [62]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [63]:
prompt_template = lambda comment: f"""[INST]JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. \
It responsts to queries aptly and ends responses with its signature '–JP_Historian'. \
JP Historian already knows many things, but it will also be given a context over the query.  \
JP Historian tries to provide comprehensive responses, yet conscise and only contain relevant information thus keeping the interaction natural and easy.

Please respond to the following query. Use the context above if it is helpful.

{comment}
[/INST]
"""

In [64]:
prompt_template_w_context = lambda context, comment: f"""[INST]JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. \
It responsts to queries aptly and ends responses with its signature '–JP_Historian'. \
JP Historian already knows many things, but it will also be given a context over the query. If the context was relevant and useful, it may use it to generate its response. \
It tries to provide comprehensive responses, yet conscise and only contain relevant information thus keeping the interaction natural and easy.

{context}

Please respond to the following query. Use the context above if it is helpful.

{comment}
[/INST]
"""

In [227]:
normal_template = lambda context, comment: f"""[INST]
JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. \
use the context below if it is helpful.

{context}

Please respond to the following query. Use the context above if it is helpful, but rephrase them, don't include them directrly.

{comment}
[/INST]
"""

In [66]:
nocon_template = lambda comment: f"""[INST]
JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. \
Please respond to the following query. 
{comment}
[/INST]
"""

In [67]:
promptWcontext, promptNcontext = prompt_template_w_context(context, query), prompt_template(query)
normalPrompt = normal_template(context, query)
noconPrompt = nocon_template(query)

In [74]:

inputs = tokenizer(promptNcontext, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


<｜begin▁of▁sentence｜>[INST]JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. It responsts to queries aptly and ends responses with its signature '–JP_Historian'. JP Historian already knows many things, but it will also be given a context over the query.  JP Historian tries to provide comprehensive responses, yet conscise and only contain relevant information thus keeping the interaction natural and easy.

Please respond to the following query. Use the context above if it is helpful.

who won the siege of osaka?
[/INST]

I'm sorry, but I couldn't find any information about the history of the siege of Osaka. Is there anything else I can help you with?
<|EOT|>


In [73]:

inputs = tokenizer(normalPrompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<｜begin▁of▁sentence｜>[INST]
JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. use the context below if it is helpful.

Context:
{"who won the siege of osaka?": "{\"title\": \"Siege of Akasaka\", \"paragraphs\": \"The  siege of Akasaka  was one of the earlier battles of the  Genk\\u014d War  between the figurehead  Emperor   Godaigo  and the largely  H\\u014dj\\u014d -controlled  Kamakura shogunate  during the final years of the  Kamakura period  in  Japan .

{"who won the siege of osaka?": "Hideyoshi's grave, along with Kyoto's  Toyokuni Shrine , were destroyed subsequently during the Tokugawa shogunate. Ch\\u014dsokabe Morichika was beheaded on May 11. There are also records of pillaging and mass rapes by Tokugawa forces at the closing of the siege. [ citation needed ] The  bakufu  obtained 650,000  koku  at  Osaka  and started rebuilding Osaka Castle. Osaka was then made a  han  (feudal domain), and given to  Matsudaira Tadayoshi ."}


## Create a custom retriver class

In [234]:
class RAGModule():
  
    def __init__(self, docs_path, embedding_model = "BAAI/bge-small-en-v1.5",
                 chunk_size = 192, chunk_overlap=24,
                 keyword_topk = 15, vector_topk = 5):
        self.initialize_settings(embedding_model, chunk_size, chunk_overlap)
        
        reader = SimpleDirectoryReader(docs_path)
        documents = reader.load_data()
        self.keyword_index = SimpleKeywordTableIndex.from_documents(documents)
        # self.vector_index = VectorStoreIndex.from_documents(documents)
        self.keyword_retriever = KeywordTableSimpleRetriever(keyword_index, similarity_top_k=keyword_topk)

    def initialize_settings(self, embedding_model, chunk_size, chunk_overlap):
        Settings.embed_model = HuggingFaceEmbedding(model_name=embedding_model)
        Settings.llm = None
        Settings.chunk_size = 192
        Settings.chunk_overlap = 24

    def apply_semantic_search(self, source_path, query):
        vsi = SimpleDirectoryReader("/Users/mehrad/research/self/codes/data/topdocs")
        top_docs = vsi.load_data()
        vsi_index = VectorStoreIndex.from_documents(top_docs)

        vector_retriever = VectorIndexRetriever(
                    index=vsi_index,
                    similarity_top_k=5)
        query_engine = RetrieverQueryEngine(
                    retriever=vector_retriever,
        )

        semantic_response = query_engine.query(query)
        return semantic_response

    def retrieve(self, query, dest_path = f"/Users/mehrad/research/self/codes/data/topdocs"):
        result = self.keyword_retriever.retrieve(query)
        top_results = []
        for x in result:
            top_results.append({query:x.text})
        for i,x in enumerate(top_results):
          with open(f"{dest_path}/{i}.json", "w") as f:
              json.dump(x, f)

        semantic_response = self.apply_semantic_search(dest_path, query)

        context_texts = [x.text for x in semantic_response.source_nodes]
        context = "Context:\n"
        for i in range(min(5, len(context_texts))):
            context = context + semantic_response.source_nodes[i].text + "\n\n"
        return context

    

In [235]:
rag = RAGModule(docs_path = f"{base_dir}/docs/")

LLM is explicitly disabled. Using MockLLM.


In [236]:
query = input()

 battle of miyajima


In [237]:
info = rag.retrieve(query = query)

In [238]:
info

'Context:\n{"battle of miyajima": "In 1554,  M\\\\u014dri Motonari , as a vassal of the \\\\u014cuchi clan, wanted to avenge the betrayed Yoshitaka, and so he rebelled against Sue, whose territorial ambitions were depleting clan resources.  The heavily outnumbered force under M\\\\u014dri attacked and defeated Sue at the  Battle of Oshikibata .  M\\\\u014dri then departed from the mainland to build a fort, known as  Miyao Castle , on Miyajima while proclaiming publicly his woe that it would not hold out long against an attack. [3] Miyao Castle was built on a hill near  Itsukushima Shrine  and facing the mainland, making it a visible and tempting target."}\n\n{"battle of miyajima": "[5]   Sue Harukata escaped from the confines of Miyao Castle, but when he saw that escape from the island was not possible, he also committed suicide by  seppuku . [2] [6]  The \\\\u014cuchi clan\'s senior vassal  Hironaka Takakane  fought, together with his son, to the end, and he was also killed. [7] The S

In [239]:
promptWcontext, promptNcontext = prompt_template_w_context(info, query), prompt_template(query)
normalPrompt = normal_template(info, query)
noconPrompt = nocon_template(query)

In [240]:
inputs = tokenizer(promptNcontext, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask = inputs['attention_mask'], max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


<｜begin▁of▁sentence｜>[INST]JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. It responsts to queries aptly and ends responses with its signature '–JP_Historian'. JP Historian already knows many things, but it will also be given a context over the query.  JP Historian tries to provide comprehensive responses, yet conscise and only contain relevant information thus keeping the interaction natural and easy.

Please respond to the following query. Use the context above if it is helpful.

battle of miyajima
[/INST]

I'm sorry, but I couldn't find any information about the Battle of Miyajima. Is there anything else I can help you with?
<|EOT|>


In [241]:
inputs = tokenizer(normalPrompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask = inputs['attention_mask'], max_new_tokens=800)

print(tokenizer.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


<｜begin▁of▁sentence｜>[INST]
JP Historian, functioning as a Japanese history expert, communicates in clear, accessible language. use the context below if it is helpful.

Context:
{"battle of miyajima": "In 1554,  M\\u014dri Motonari , as a vassal of the \\u014cuchi clan, wanted to avenge the betrayed Yoshitaka, and so he rebelled against Sue, whose territorial ambitions were depleting clan resources.  The heavily outnumbered force under M\\u014dri attacked and defeated Sue at the  Battle of Oshikibata .  M\\u014dri then departed from the mainland to build a fort, known as  Miyao Castle , on Miyajima while proclaiming publicly his woe that it would not hold out long against an attack. [3] Miyao Castle was built on a hill near  Itsukushima Shrine  and facing the mainland, making it a visible and tempting target."}

{"battle of miyajima": "[5]   Sue Harukata escaped from the confines of Miyao Castle, but when he saw that escape from the island was not possible, he also committed suicide by